In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama

In [29]:
prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um humorista e faz piadas conforme solicitado pelo usuário.'),
    ('human', 'Faça uma piada sobre o tema: {tema}')
])
model = Ollama(model='llama3.1')
parser = StrOutputParser()

chain = prompt | model | parser

chain.input_schema.schema()

{'properties': {'tema': {'title': 'Tema', 'type': 'string'}},
 'required': ['tema'],
 'title': 'PromptInput',
 'type': 'object'}

In [20]:
chain.invoke({'tema': 'astronautas'})

'Que espaço para rir!\n\nPor que os astronautas sempre vão ao psicólogo? Porque estão "outros" em crise! (saudades do espaço são real!)\n\nMas sério, por que as missões espaciais sempre levam uma quantidade impressionante de... você adivinhou? Cerveja! É para manter os nervos estabilizados quando você está fazendo um pouquinho de "gravidade" no universo!\n\nE sabe o que é melhor? Quando eles voltam da missão, as pessoas perguntam: "Como foi?" E eles respondem... "Foi um \'blasta\'!" (só quem foi ao espaço entende!)\n\nO que você acha dessa piada? Quer mais?'

In [31]:
stream = chain.stream({'tema': 'astronautas'})

for chunk in stream:
    print(chunk, end="", flush=True)
print()

Por que os astronautas se sentem sempre desconfortáveis? Porque estão sempre fora do seu elemento!

(Por favor, diga que gostou ou quiser mais!)


In [32]:
prompt = ChatPromptTemplate([
    ('system', 'You are a comedian and make jokes as requested by the user.'),
    ('human', 'Tell me a joke about the topic: {topic}')
])

model = Ollama(model='llama3.1', temperature=0.1)

parser = StrOutputParser()

chain = prompt | model | parser

print(chain.input_schema.schema())

topic = input('Enter a topic for the question: ')

stream = chain.stream({'topic': topic})
for chunk in stream:
    print(chunk, end="", flush=True)
print()

{'properties': {'topic': {'title': 'Topic', 'type': 'string'}}, 'required': ['topic'], 'title': 'PromptInput', 'type': 'object'}
Here's one:

Why did the astronaut break up with his girlfriend before going to Mars?

Because he needed space! (get it?)


In [33]:
from langchain_core.prompts import PromptTemplate

prompt_translate = PromptTemplate.from_template("Translate the following piece of text into portuguese (BR). Provide the translation ONLY. \n {text}")

# model_translate = Ollama(model="llama3.1", temperature = 0.1)

chain_translate = prompt_translate | model | StrOutputParser()

print(chain_translate.invoke({'text': 'Be nice to people all the time, Jeremy'}))

Seja amável com as pessoas o tempo todo, Jeremy.


In [24]:
print(chain.output_schema.schema())

print(chain_translate.input_schema.schema())

{'title': 'StrOutputParserOutput', 'type': 'string'}
{'properties': {'text': {'title': 'Text', 'type': 'string'}}, 'required': ['text'], 'title': 'PromptInput', 'type': 'object'}


In [25]:
from langchain_core.runnables import RunnablePassthrough

composed_chain = chain | {'text': RunnablePassthrough()} | chain_translate

print(composed_chain.invoke({'topic': 'astronaut'}))

Por que o astronauta se separou da namorada antes de ir para Marte?

Porque ele precisava de espaço! (entendeu?)


In [27]:
prompt_input = PromptTemplate.from_template("Translate the following portuguese topic into english. Provide ONLY the translation. \n {topic}")

composed_chain = (
    prompt_input 
    | model 
    | StrOutputParser()
    | {'topic': RunnablePassthrough()}
    | chain
    | {'text': RunnablePassthrough()}
    | chain_translate
)

stream = composed_chain.stream({'topic', 'Astronauta'})
for chunk in stream:
    print(chunk, end="", flush=True)
print()

Por que o astronauta se separou da namorada antes de ir para Marte?

Porque ele precisava de espaço! (entendeu?)
